In [1]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle"
sub_path = "../../data/tencent2020/sub"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)

In [2]:
user_creative_ids = pd.read_pickle(f"{pickle_path}/user_creative_ids.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_creative_ids = user_creative_ids.merge(user, how='left', on='user_id')
user_creative_ids.head()

,user_id,creative_id,age,gender
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...",4.0,1.0
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...",10.0,1.0
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...",7.0,2.0
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...",5.0,1.0
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...",4.0,1.0


In [ ]:
# Gen W2V Vector
from gensim import models

for i,df in enumerate([active,usage]): # 预计耗时 1511 + 5874
    t1 = time.time()
    w2v = models.Word2Vec(df['appid'].values, size=300, window=20, workers=40,hs=1) # 设置sg的话 变成skip-gram方法 我们测试效果差不多
    w2v.wv.save_word2vec_format("{}/{}_w2v300.model".format(vector_path,i))
    print(time.time()-t1)